Primero vamos a proceder importando las librerias que vamos a usar en el desarrollo de la práctica

In [65]:
import glob
import cv2
from skimage import io, transform, color
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot
import requests
import os

Para comenzar con el preprocesamiento de las imagenes primero vamos a clonar el repositorio en el que estan las imagenes para poder trabajar en local.

In [61]:
os.system("git clone https://github.com/Kekon130/ML_Practica3.git") 

32768

Ahora vamos a meter las imagenes en un array para poder acceder de forma fácila a ellas.

In [81]:
ruta_train = "./ML_Practica3/train"
ruta_test = "./ML_Practica3/test"

df_train = []
img_size = 80

for img in os.listdir(ruta_train):
  img_normal = io.imread(os.path.join(ruta_train, img))
  img_resize = transform.resize(img_normal, output_shape=(img_size, img_size))
  img_gray = color.rgb2gray(img_resize)
  df_train.append(img_gray)